### 강우 기반 파생

#### 강수 변화율·집중도

- ΔRN_15m, ΔRN_60m, ΔRN_12H

- RN_15m / RN_60m (단기 집중도)

#### 누적 강수 메모리(Antecedent Rainfall / AR)

- AR_3H, AR_6H, AR_12H, AR_24H = Σ RN_15m(최근 x시간)

- log(1 + AR_x)

#### 무강수 지속시간 + first flush

- dry_duration_h (마지막 강수 이후 경과 시간)

- rain_start / rain_end 플래그

- post_rain_6H 플래그(종료 후 잔류 효과)

#### API 지수(감쇠 누적)

- API(RN_15m, k, N) = Σ RN_15m(t−i)·exp(−k·i)

In [60]:
import pandas as pd
import numpy as np

In [61]:
def _add_rain_features(X, station_ids, rain_cols, rule, antecedent_hours, wet_thr_mm, eps, add_api, api_k, api_hours):
    """강수 관련 특성 생성"""
    new_cols = {}
    
    # steps_per_hour 계산
    steps_per_hour = int(pd.Timedelta("1h") / pd.Timedelta(rule))
    
    # Delta (변화율) - 시간을 스텝으로 변환
    delta_map = {"RN_15m": 3, "RN_60m": 12, "RN_12H": 144}  # 5분 기준 스텝 수
    for sid in station_ids:
        for rc in rain_cols[:3]:  # RN_15m, RN_60m, RN_12H만
            col = f"{rc}_{sid}"
            if col in X.columns:
                new_cols[f"d_{col}"] = X[col].diff(delta_map[rc])
    
    # Intensity ratio (단기 집중도)
    for sid in station_ids:
        rn15 = f"RN_15m_{sid}"
        rn60 = f"RN_60m_{sid}"
        if rn15 in X.columns and rn60 in X.columns:
            new_cols[f"RN_15m_div_RN_60m_{sid}"] = X[rn15] / (X[rn60] + eps)
    
    # Antecedent rainfall (누적 강수)
    for sid in station_ids:
        col = f"RN_15m_{sid}"
        if col in X.columns:
            for h in antecedent_hours:
                win = h * steps_per_hour
                ar_col = X[col].rolling(win, min_periods=max(2, win // 10)).sum()
                new_cols[f"AR_{h}H_{sid}"] = ar_col
                new_cols[f"log1p_AR_{h}H_{sid}"] = np.log1p(ar_col)
    
    # Dry duration + first flush
    for sid in station_ids:
        col = f"RN_15m_{sid}"
        if col in X.columns:
            wet = (X[col].fillna(0) >= wet_thr_mm)
            last_wet_ts = pd.Series(X.index.where(wet), index=X.index).ffill()
            dry_timedelta = (X.index - last_wet_ts)
            
            new_cols[f"dry_duration_min_{sid}"] = dry_timedelta.dt.total_seconds().fillna(0) / 60.0
            new_cols[f"dry_duration_hr_{sid}"] = dry_timedelta.dt.total_seconds().fillna(0) / 3600.0
            new_cols[f"is_wet_{sid}"] = wet.astype(np.int8)
            
            # First flush 힌트
            rain_start = wet & (~wet.shift(1, fill_value=False))
            rain_end = (~wet) & (wet.shift(1, fill_value=False))
            new_cols[f"rain_start_{sid}"] = rain_start.astype(np.int8)
            new_cols[f"rain_end_{sid}"] = rain_end.astype(np.int8)
            
            # 종료 후 6시간 잔류
            post_win = 6 * steps_per_hour
            new_cols[f"post_rain_6H_{sid}"] = (
                pd.Series(rain_end.values, index=X.index)
                .rolling(post_win, min_periods=1).max()
                .fillna(0).astype(np.int8)
            )
    
    # API (감쇠 누적) - 선택적
    if add_api:
        api_steps = api_hours * steps_per_hour
        weights = np.exp(-api_k * np.arange(1, api_steps + 1, dtype=np.float32))
        for sid in station_ids:
            col = f"RN_15m_{sid}"
            if col in X.columns:
                rain = X[col].to_numpy(dtype=np.float32)
                api = np.full_like(rain, np.nan, dtype=np.float32)
                for t in range(len(rain)):
                    start = max(0, t - api_steps)
                    seg = rain[start:t]
                    if seg.size == 0:
                        api[t] = 0.0
                    else:
                        w = weights[-seg.size:]
                        api[t] = float(np.sum(seg[::-1] * w))
                new_cols[f"API_RN_15m_k{api_k}_H{api_hours}_{sid}"] = api
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X


In [62]:
def _add_areal_rain_features(X, station_ids, rain_cols, eps=1e-6):
    """관측소 강수를 면적강수(평균/최대)로 요약"""
    new_cols = {}

    for rc in rain_cols:
        cols = [f"{rc}_{sid}" for sid in station_ids if f"{rc}_{sid}" in X.columns]
        if not cols:
            continue

        # 평균/최대 면적강수
        new_cols[f"{rc}_areal_mean"] = X[cols].mean(axis=1)
        new_cols[f"{rc}_areal_max"]  = X[cols].max(axis=1)

        # 공간 불일치(국지성) 지표: max-mean, std
        new_cols[f"{rc}_areal_max_minus_mean"] = new_cols[f"{rc}_areal_max"] - new_cols[f"{rc}_areal_mean"]
        new_cols[f"{rc}_areal_std"] = X[cols].std(axis=1, ddof=0)

    # 면적강수 집중도(areal)
    if "RN_15m_areal_mean" in new_cols and "RN_60m_areal_mean" in new_cols:
        new_cols["RN_15m_div_RN_60m_areal_mean"] = new_cols["RN_15m_areal_mean"] / (new_cols["RN_60m_areal_mean"] + eps)

    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X


### 기상 파생

#### 열·습 결합

- DewPointDepression = TA − TD (추천)

(선택) VPD proxy / HeatIndex (효과는 케이스바이케이스 → uncertain)

#### 기상 안정성(변동성)

- rolling_std(TA, 3H/6H)

- rolling_std(HM, 3H/6H)

In [63]:
def _add_weather_features(X, station_ids, weather_cols, rule, stability_windows, eps):
    """기상 관련 특성 생성"""
    new_cols = {}
    steps_per_hour = int(pd.Timedelta("1h") / pd.Timedelta(rule))
    
    for sid in station_ids:
        ta_col = f"TA_{sid}"
        td_col = f"TD_{sid}"
        hm_col = f"HM_{sid}"
        
        # Dew point depression
        if ta_col in X.columns and td_col in X.columns:
            new_cols[f"TA_minus_TD_{sid}"] = X[ta_col] - X[td_col]
        
        # VPD 
        if ta_col in X.columns and hm_col in X.columns:
            T = X[ta_col]
            RH = X[hm_col].clip(0, 100)
            e_s = 0.6108 * np.exp((17.27 * T) / (T + 237.3))
            new_cols[f"VPD_kPa_{sid}"] = e_s * (1 - RH / 100.0)
        
        # Stability (rolling std)
        for c in [ta_col, hm_col]:
            if c not in X.columns:
                continue
            for w in stability_windows:
                win_steps = int(pd.Timedelta(w) / pd.Timedelta(rule))
                minp = max(2, int(win_steps * 0.1))
                new_cols[f"{c}_std_{w}"] = X[c].rolling(win_steps, min_periods=minp).std(ddof=0)
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X

### 시계열 메모리: lag & rolling

#### Lag features

추천 lag:

- 10min, 30min, 60min

적용 컬럼:

- PH_VU, FLUX_VU, SS_VU, TOC_VU, TA, RN_15m(또는 RN_60m)

- (선택) TN_lag, TP_lag (누수 아님)

#### Rolling 통계량

윈도우:

- 30min, 60min, 3H

통계:

- rolling_mean, rolling_std, rolling_max (강우는 rolling_sum도 추가)

적용 컬럼:

- PH_VU, FLUX_VU, SS_VU, TOC_VU, RN_15m, TA/HM

#### 변화율(derivative)

- ΔPH, ΔFLUX, ΔSS, ΔTOC, ΔTA

- |ΔFLUX| (급변 여부)

In [64]:
def _add_lag_roll_delta_features(X, process_cols, station_ids, rain_cols, weather_cols, rule, lag_list, roll_windows, eps):
    """시계열 특성 (lag, rolling, delta) 생성"""
    new_cols = {}
    
    # 대상 컬럼 구성
    base_cols = list(process_cols) + [f"{wc}_{sid}" for sid in station_ids for wc in weather_cols]
    base_cols += [f"RN_15m_{sid}" for sid in station_ids]
    base_cols = [c for c in base_cols if c in X.columns]
    
    # Lag
    for c in base_cols:
        for L in lag_list:
            lag_steps = int(pd.Timedelta(L) / pd.Timedelta(rule))
            new_cols[f"{c}_lag_{L}"] = X[c].shift(lag_steps)
        
        # Delta
        new_cols[f"d_{c}"] = X[c].diff()
    
    # Abs delta (급변 여부)
    if "FLUX_VU" in X.columns:
        new_cols["abs_d_FLUX_VU"] = X["FLUX_VU"].diff().abs()
    
    # Rolling
    for c in base_cols:
        for w in roll_windows:
            win_steps = int(pd.Timedelta(w) / pd.Timedelta(rule))
            minp = max(2, int(win_steps * 0.1))
            r = X[c].rolling(win_steps, min_periods=minp)
            new_cols[f"{c}_roll_mean_{w}"] = r.mean()
            new_cols[f"{c}_roll_std_{w}"] = r.std(ddof=0)
            new_cols[f"{c}_roll_max_{w}"] = r.max()
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X

### 공정·수질 결합 특성 (누수 없이 재설계)

#### 부하(load) 계열 (아주 중요)

- SS_load = FLUX_VU × SS_VU

- TOC_load = FLUX_VU × TOC_VU

- FLUX_VU × (SS_VU + TOC_VU)

#### 상호작용(interaction)

- RN_15m × FLUX_VU

- dry_duration_h × RN_15m

- RN_60m × SS_lag (예: SS_lag_10m)

- PH_VU × TOC_VU

- SS_VU × FLUX_VU

#### 비율(ratio) 

- TOC_VU / SS_VU

In [65]:
def _add_process_features(X, process_cols, rule, spike_z, spike_window, eps):
    """공정 관련 특성 생성"""
    new_cols = {}
    
    # Load (부하)
    if all(c in X.columns for c in ["FLUX_VU", "SS_VU"]):
        new_cols["SS_load"] = X["FLUX_VU"] * X["SS_VU"]
    
    if all(c in X.columns for c in ["FLUX_VU", "TOC_VU"]):
        new_cols["TOC_load"] = X["FLUX_VU"] * X["TOC_VU"]
    
    # Interactions
    if all(c in X.columns for c in ["PH_VU", "TOC_VU"]):
        new_cols["PH_x_TOC"] = X["PH_VU"] * X["TOC_VU"]
    
    if all(c in X.columns for c in ["SS_VU", "FLUX_VU"]):
        new_cols["SS_x_FLUX"] = X["SS_VU"] * X["FLUX_VU"]
    
    # Ratios
    if all(c in X.columns for c in ["TOC_VU", "SS_VU"]):
        new_cols["TOC_div_SS"] = X["TOC_VU"] / (X["SS_VU"] + eps)
    
    # pH zone
    if "PH_VU" in X.columns:
        ph = X["PH_VU"]
        zone = pd.cut(ph, bins=[-np.inf, 6.5, 7.5, np.inf], labels=[0, 1, 2])
        new_cols["PH_zone"] = zone.astype("float").fillna(1).astype(np.int8)
    
    # Spike flags
    steps_per_hour = int(pd.Timedelta("1h") / pd.Timedelta(rule))
    win_steps = int(pd.Timedelta(spike_window) / pd.Timedelta(rule))
    minp = max(5, int(win_steps * 0.1))
    
    spike_cols = ["SS_VU", "TOC_VU", "PH_VU", "FLUX_VU"]
    for c in spike_cols:
        if c not in X.columns:
            continue
        mu = X[c].rolling(win_steps, min_periods=minp).mean()
        sd = X[c].rolling(win_steps, min_periods=minp).std(ddof=0)
        z = (X[c] - mu) / (sd + eps)
        new_cols[f"{c}_spike_z{spike_z:g}"] = (z > spike_z).astype(np.int8)
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X

In [66]:
def _add_rain_process_interactions(X, station_ids, eps):
    """강수-공정 상호작용 특성 생성"""
    new_cols = {}
    
    for sid in station_ids:
        rn15 = f"RN_15m_{sid}"
        rn60 = f"RN_60m_{sid}"
        dry = f"dry_duration_hr_{sid}"
        
        if "FLUX_VU" in X.columns and rn15 in X.columns:
            new_cols[f"RN_15m_x_FLUX_{sid}"] = X[rn15] * X["FLUX_VU"]
        
        if dry in X.columns and rn15 in X.columns:
            new_cols[f"dry_x_RN_15m_{sid}"] = X[dry] * X[rn15]
        
        if rn60 in X.columns and f"{rn15}_lag_10min" in X.columns:
            new_cols[f"RN_60m_x_RN15lag10_{sid}"] = X[rn60] * X[f"{rn15}_lag_10min"]
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    return X


### 시간(주기) 특성 (안 넣으면 손해)

- hour_sin, hour_cos

- is_weekend

In [67]:
def _add_time_features(X):
    """시간 관련 특성 생성"""
    new_cols = {}
    
    new_cols["hour"] = X.index.hour.astype(np.int16)
    new_cols["dow"] = X.index.dayofweek.astype(np.int8)
    new_cols["is_weekend"] = (X.index.dayofweek >= 5).astype(np.int8)
    
    # Cyclical encoding
    hour_values = X.index.hour
    new_cols["hour_sin"] = np.sin(2 * np.pi * hour_values / 24.0)
    new_cols["hour_cos"] = np.cos(2 * np.pi * hour_values / 24.0)

    # Cyclical encoding (연중)
    doy = X.index.dayofyear
    new_cols["doy_sin"] = np.sin(2 * np.pi * doy / 365.25)
    new_cols["doy_cos"] = np.cos(2 * np.pi * doy / 365.25)
    
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
    
    return X

In [68]:
def resample_5min(
    df,
    time_col=None,
    rule="5min",
    sum_cols=None,
    mean_cols=None,
    extra_mean_cols=(),
    interp_limit: int = 12,
):
    """
    1분(or irregular) -> 5분 리샘플링.
    """
    x = df.copy()

    if time_col is not None:
        x[time_col] = pd.to_datetime(x[time_col])
        x = x.set_index(time_col)
    if not isinstance(x.index, pd.DatetimeIndex):
        raise ValueError("df must have a DatetimeIndex or provide time_col.")
    x = x.sort_index()

    # 숫자화
    all_cols = set(sum_cols or []) | set(mean_cols or []) | set(extra_mean_cols)
    for c in all_cols:
        if c in x.columns:
            x[c] = pd.to_numeric(x[c], errors="coerce")

    # 집계 dict
    agg = {}
    for c in sum_cols or []:
        if c in x.columns:
            agg[c] = "sum"
    for c in list(mean_cols or []) + list(extra_mean_cols):
        if c in x.columns:
            agg[c] = "mean"

    if not agg:
        raise ValueError("No columns found to resample.")

    # 리샘플
    out = x.resample(rule).agg(agg)

    # 결측 처리
    for c in sum_cols or []:
        if c in out.columns:
            out[c] = out[c].fillna(0.0)

    for c in list(mean_cols or []) + list(extra_mean_cols):
        if c in out.columns:
            out[c] = out[c].interpolate(method="time", limit=interp_limit)

    return out

In [69]:
def make_modelB_features(
    df,
    time_col=None,
    do_resample=True,
    rule="5min",
    station_ids=("368", "541", "569"),
    rain_cols=("RN_15m", "RN_60m", "RN_12H", "RN_DAY"),
    weather_cols=("TA", "HM", "TD"),
    process_cols=("PH_VU", "FLUX_VU", "SS_VU", "TOC_VU"),
    lag_list=("10min", "30min", "1h"),
    roll_windows=("30min", "1h", "3h"),
    antecedent_hours=(3, 6, 12, 24),
    stability_windows=("3h", "6h"),
    wet_thr_mm=0.1,
    spike_z=2.0,
    spike_window="24h",
    add_api=False,
    api_k=0.01,
    api_hours=24,
    eps=1e-6,
):
    
    # 1. 기본 컬럼 구성
    base_cols = []
    for sid in station_ids:
        base_cols.extend([f"{wc}_{sid}" for wc in weather_cols])
        base_cols.extend([f"{rc}_{sid}" for rc in rain_cols])
    base_cols.extend(process_cols)
    
    # 2. 리샘플링
    if do_resample:
        sum_cols = tuple(f"{rc}_{sid}" for sid in station_ids for rc in rain_cols) + ("FLUX_VU",)
        mean_cols = (
            tuple(f"{wc}_{sid}" for sid in station_ids for wc in weather_cols) + 
            ("PH_VU", "SS_VU", "TOC_VU")
        )
        df = resample_5min(df, time_col=time_col, rule=rule, sum_cols=sum_cols, mean_cols=mean_cols)
        time_col = None
    
    # 3. 인덱스 설정
    x = df.copy()
    if time_col is not None:
        x[time_col] = pd.to_datetime(x[time_col])
        x = x.set_index(time_col)
    if not isinstance(x.index, pd.DatetimeIndex):
        raise ValueError("df must have a DatetimeIndex or provide time_col.")
    x = x.sort_index()
    
    # 4. 컬럼 검증
    missing = [c for c in base_cols if c not in x.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    
    X = x[base_cols].copy()
    
    # 5. 강수 특성 생성
    X = _add_rain_features(X, station_ids, rain_cols, rule, antecedent_hours, wet_thr_mm, eps, add_api, api_k, api_hours)
    
    # 6. 기상 특성 생성
    X = _add_weather_features(X, station_ids, weather_cols, rule, stability_windows, eps)
    
    # 7. 시계열 특성 (lag, rolling, delta)
    X = _add_lag_roll_delta_features(X, process_cols, station_ids, rain_cols, weather_cols, rule, lag_list, roll_windows, eps)
    
    # 8. 공정 특성 생성
    X = _add_process_features(X, process_cols, rule, spike_z, spike_window, eps)
    
    # 9. 강수-공정 상호작용
    X = _add_rain_process_interactions(X, station_ids, eps)
    
    # 10. 시간 특성
    X = _add_time_features(X)
    
    # 11. 정리
    X = X.replace([np.inf, -np.inf], np.nan)
    
    return X

In [70]:
tms = pd.read_csv("../../data/processed/TMS_cleaned.csv")
aws = pd.read_csv("../../data/processed/AWS_cleaned.csv")

tms['SYS_TIME'] = pd.to_datetime(tms['SYS_TIME'])
aws['SYS_TIME'] = pd.to_datetime(aws['SYS_TIME'])

tms = tms.sort_values('SYS_TIME')
aws = aws.sort_values('SYS_TIME')

df = pd.merge_asof(tms, aws, on='SYS_TIME', direction='backward', tolerance=pd.Timedelta('1min'))

In [71]:
X = make_modelB_features(df, time_col="SYS_TIME", do_resample=True, rule="5min")

y_tn = resample_5min(df, time_col="SYS_TIME", extra_mean_cols=("TN_VU","TP_VU"))["TN_VU"].shift(-1)
y_tp  = resample_5min(df, time_col="SYS_TIME", extra_mean_cols=("TN_VU","TP_VU"))["TP_VU"].shift(-1)

data = X.join(pd.DataFrame({"y_tn": y_tn, "y_tp": y_tp})).dropna()

In [72]:
data

,TA_368,HM_368,TD_368,RN_15m_368,RN_60m_368,RN_12H_368,RN_DAY_368,TA_541,HM_541,TD_541,...,RN_60m_x_RN15lag10_569,hour,dow,is_weekend,hour_sin,hour_cos,doy_sin,doy_cos,y_tn,y_tp
SYS_TIME,,,,,,,,,,,,,,,,,,,,,
2024-08-27 03:05:00,26.10,80.12,22.40,0.0,0.0,2.5,0.0,26.06,84.08,23.16,...,0.0,3,1,0,0.707107,0.707107,-0.834370,-0.551205,6.468,0.077
2024-08-27 03:10:00,26.08,79.84,22.32,0.0,0.0,2.5,0.0,25.96,84.14,23.06,...,0.0,3,1,0,0.707107,0.707107,-0.834370,-0.551205,6.468,0.077
2024-08-27 03:15:00,26.00,79.62,22.20,0.0,0.0,2.5,0.0,25.90,84.52,23.10,...,0.0,3,1,0,0.707107,0.707107,-0.834370,-0.551205,6.468,0.077
2024-08-27 03:20:00,25.98,79.38,22.14,0.0,0.0,2.5,0.0,25.78,85.10,23.10,...,0.0,3,1,0,0.707107,0.707107,-0.834370,-0.551205,6.468,0.077
2024-08-27 03:25:00,25.90,79.14,22.00,0.0,0.0,2.5,0.0,25.76,85.02,23.06,...,0.0,3,1,0,0.707107,0.707107,-0.834370,-0.551205,6.468,0.077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-29 04:55:00,17.48,81.50,14.28,0.0,0.0,0.0,0.0,16.24,95.42,15.52,...,0.0,4,0,0,0.866025,0.500000,-0.999445,-0.033324,7.645,0.024
2025-09-29 05:00:00,17.40,82.10,14.32,0.0,0.0,0.0,0.0,16.26,95.74,15.56,...,0.0,5,0,0,0.965926,0.258819,-0.999445,-0.033324,7.645,0.024
2025-09-29 05:05:00,17.40,82.70,14.40,0.0,0.0,0.0,0.0,16.20,95.76,15.50,...,0.0,5,0,0,0.965926,0.258819,-0.999445,-0.033324,7.645,0.024


In [73]:
data.to_csv("../../data/processed/modelB_dataset.csv", index=True)